# **Predicting the XRP Price using Geometric Brownian Motion for 30 days**

In [18]:
import pandas as pd
import numpy as np

In [19]:
import plotly.graph_objects as go
from plotly.offline import plot

In [37]:
from scipy import stats

In [20]:
# Load stock prices from CSV
file_path = "C:\\Heri\\GitHub\\Crypto\\02-Coingecko\\data-crypto.csv"
data = pd.read_csv(file_path)

In [21]:
# Assuming the CSV has a column named 'Price'
prices = data['average']  # Replace with your column name
dates = data['timestamp']  # Optional, if you want to visualize dates

In [22]:
# Calculate daily returns
daily_returns = prices.pct_change().dropna()

In [23]:
# Estimate parameters for Brownian motion
mu = daily_returns.mean()  # Mean return
sigma = daily_returns.std()  # Volatility

## **Geometric Brownian Motion**

In [24]:
# Parameters for simulation
start_price = prices.iloc[-1]  # Start from the last known price
time_horizon = 365  # Days
steps = 3650  # Simulation steps

In [25]:
# Simulate future prices using geometric Brownian motion
def simulate_gbm(start_price, mu, sigma, time_horizon, steps):
    dt = time_horizon / steps
    price_path = [start_price]
    for _ in range(steps):
        price_path.append(price_path[-1] * np.exp((mu - 0.5 * sigma ** 2) * dt +
                                                  sigma * np.sqrt(dt) * np.random.normal()))
    return price_path

In [26]:
# Run simulation
simulated_prices = simulate_gbm(start_price, mu, sigma, time_horizon, steps)

In [27]:
# Create the plot
fig = go.Figure()

# Add the simulated prices line
fig.add_trace(go.Scatter(
    x=list(range(len(simulated_prices))),
    y=simulated_prices,
    mode='lines',
    name='Simulated Prices',
    line=dict(color='#206583', width=2)
))

# Update layout for aesthetics
fig.update_layout(
    title={
        'text': 'Average Crypto Market Price Prediction using GBM for 1 Year',
        'font': {
            'size': 30
        }
    },
    xaxis_title='Time Step',
    yaxis_title='Price',
    font=dict(size=14),
    plot_bgcolor='white',
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey')
)

In [28]:
# Render the plot in HTML
plot(fig, auto_open=False, filename='Average Crypto Market Price Prediction.html')

'Average Crypto Market Price Prediction.html'

## **Multiple Geometric Brownian Motion**

In [29]:
# Run simulation 10 times
simulations = []

simulations.append(simulated_prices)

for simulation in range(14):
    simulated_prices = simulate_gbm(start_price, mu, sigma, time_horizon, steps)
    simulations.append(simulated_prices)

In [30]:
# Create the plot
fig = go.Figure()

# Define a list of colors
colors = ['#0D3239', '#1A5465', '#206583', '#3487AB', '#3487AB', '#0D3239', '#113C46', '#184F5F', '#1D5B72', '#1E607A', '#266F8E', '#3182A5', '#3487AB', '#3487AB', '#3487AB']

# Add each simulated prices line with different colors
for i, prices in enumerate(simulations):
    fig.add_trace(go.Scatter(
        x=list(range(len(prices))),
        y=prices,
        mode='lines',
        name=f'Simulation {i+1}',
        line=dict(color=colors[i % len(colors)], width=1)
    ))

# Update layout for aesthetics
fig.update_layout(
    title={
        'text': 'Average Crypto Market Price Prediction using GBM for 1 Year',
        'font': {
            'size': 30
        }
    },
    xaxis_title='Time Step',
    yaxis_title='Price',
    font=dict(size=14),
    plot_bgcolor='white',
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey')
)

In [31]:
# Render the plot in HTML
plot(fig, auto_open=False, filename='Average Crypto Market Price Multiple Prediction using GBM for 1 Year.html')

'Average Crypto Market Price Multiple Prediction using GBM for 1 Year.html'

## **Probability of the Last Price Being Greater than the Initial Price**

In [32]:
for simulation in range(2000):
    simulated_prices = simulate_gbm(start_price, mu, sigma, time_horizon, steps)
    simulations.append(simulated_prices)

In [33]:
# Get last values
final_values = [sim[-1] for sim in simulations]

# Get initial value
initial_value = simulations[0][0]

In [34]:
count_greater = sum(1 for value in final_values if value > initial_value)
total_values = len(final_values)
probability = count_greater / total_values if total_values > 0 else 0

print(f'Initial Value: {initial_value}')
print(f'Count Greater: {count_greater}')
print(f'Total Values: {total_values}')
print(f'Probability: {probability}')


Initial Value: 4606.794136604933
Count Greater: 1889
Total Values: 2015
Probability: 0.937468982630273


## **Distribution of the Last Price using Geometric Brownian Motion**

In [35]:
# Convert list to pandas Series
data_series = pd.Series(final_values)
print(data_series.describe())

count     2015.000000
mean     11640.351052
std       6563.993020
min       1607.208112
25%       7059.542490
50%      10020.851059
75%      14449.512840
max      60223.265976
dtype: float64


In [38]:
# Fit a gamma distribution to the data
shape, loc, scale = stats.gamma.fit(data_series)

# Print the estimated parameters
print(f"Shape: {shape}, Location: {loc}, Scale: {scale}")

Shape: 38.81481140206006, Location: -23702.00366677894, Scale: 904.0583800569775


In [39]:
# Calculate statistics
mean = np.mean(final_values)
std = np.std(final_values)
skew = stats.skew(final_values)
kurtosis = stats.kurtosis(final_values)
median = np.median(final_values)

In [41]:
# Create histogram
hist_data = go.Histogram(
    x=final_values,
    nbinsx=200,
    histnorm='probability density',
    marker=dict(color='skyblue', opacity=0.7),
    name='Histogram'
)

# Add normal distribution curve
x = np.linspace(0, max(final_values), 100)

# Calculate the gamma distribution curve
gamma_curve = go.Scatter(
    x=x,
    y=stats.gamma.pdf(x, shape, loc, scale),
    mode='lines',
    line=dict(color='blue', width=2),
    name='Gamma Distribution'
)

# Add vertical line for initial value
initial_value_line = go.Scatter(
    x=[initial_value, initial_value],
    y=[0, max(stats.norm.pdf(x, mean, std))],
    mode='lines',
    line=dict(color='green', width=2, dash='dash'),
    name='Initial Value'
)

# Create figure
fig = go.Figure(data=[hist_data, gamma_curve, initial_value_line])

# Add statistics annotation
fig.add_annotation(
    xref='paper', yref='paper',
    x=0.05, y=0.95,
    text=f'Mean: {mean:.2f}<br>Std: {std:.2f}<br>Skew: {skew:.2f}<br>Kurtosis: {kurtosis:.2f}<br>Median: {median:.2f}',
    showarrow=False,
    align='left'
)

# Update layout
fig.update_layout(
    title={
        'text': 'Distribution of Final Values for XRP in 30 Days',
        'font': {
            'size': 30
        }
    },
    xaxis_title='Final Price',
    yaxis_title='Density',
    legend=dict(x=0.7, y=0.95),
    plot_bgcolor='white',
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey')
)

In [ ]:
# Render the plot in HTML
plot(fig, auto_open=False, filename='XRP Distribution using GBM.html')

'XRP Distribution using GBM.html'